In [ ]:
import cv2
import numpy as np

import os
import sys

# Location of frames
training_image_src = '/mnt/disks/a/frames'
validation_image_src = '/mnt/disks/b/frames'

"""
The directory is divided into folders by the candidate number
Within each candidate's folder, the frames are further divided by the label
Naming convention of the frames is as follows: [candidate number]_[frame_number]_[label]
Single digit candidate numbers are padded with a 0
Frame numbers are consecutive and not padded
Label can be 0, 5 or 10
"""

# This function helps to extract data and labels and return it as a Numpy array from a given image file
def extract_data_and_label(image_path):
    # We use opencv to read the images as grayscale, this will give us the 2d vector of pixels
    # Note that it returns a numpy array and not a Python list, but Keras uses Numpy arrays anyway
    image = cv2.imread(image_path, cv2.cv2.IMREAD_GRAYSCALE)
    # Because some of the images are corrupt, we got to do this
    if image is None or image.data is None or image.size == 0:
        return None, None

    # Scale the images to a fixed size, second argument is the target dimension, chose an arbitrary
    # value for now, (100, 100). Additional arguments can be provided to fine-tune the scaling.
    image = cv2.resize(image, (100, 100))
    image = image / 255

    """
    !!! Should we extract only the faces? By right CNN is supposed to be able to pick out key features
    on its own, but this could possibly make it more effective. This can be done using opencv
    """

    # Next is to extract the labels for each image, in our case, it is just the last portion of the filename
    file_name = os.path.basename(image_path)
    label = int(os.path.splitext(file_name)[0].split('_')[2])
    # Convert to 0, 1 - we are only using images with labels 0 and 10 now
    label = 0 if label == 0 else 1
    
    return image, label

# Time to actually train the model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.utils import to_categorical
from keras.models import model_from_json
from keras.regularizers import l1
from keras.losses import BinaryCrossentropy

"""
Gonna have to do the procesing in batches because the images are too big to fit all on the ram at the same time.
To do so, we define a generator function that will help pull data in batches from the disks.

https://mc.ai/train-keras-model-with-large-dataset-batch-training/
"""
def batch_generator(files, batch_size):
    counter = 0
    while True:
        pixels = []
        labels = []

        # print('Generating batch...')
        while len(pixels) < batch_size:
            filename = files[counter]
            data, label = extract_data_and_label(filename)
            
            if data is None and label is None:
                counter = (counter + 1) % len(files)
                continue

            counter = (counter + 1) % len(files)
            pixels.append(data)
            labels.append(label)

        pixels = np.array(pixels)
        labels = np.array(labels)

        """
        Gotta reformat the data (once again) to a format that the Conv2D layer accepts. Conv2D layer
        is just the convulutional layer provided by keras.

        The target format is (w, x, y, z) where w is the number of total images, x and y is the shape of each image
        and z is 1 which signifies that the images are grayscale
        """
        pixels = pixels.reshape(batch_size, 100, 100, 1)

        """
        We one-hot-encode our labels to create 3 cateogories, 0 being mapped awake, 5 being mapped to normal and 10 being
        mapped to sleepy

        !!! Perhaps there can be a better way of encoding the output data? Will this method result in a loss of ordinality?
        """
        labels = to_categorical(labels, num_classes=2)
        # yield is a Python thing for generators
        yield pixels, labels
    

# Let's instantiate our generators for the training and validation set
print('Creating generators...')
training_files = []
for root, dirs, files in os.walk(training_image_src):
    for file in files[:300]:
        file_path = os.path.join(root, file)
        training_files.append(file_path)      

training_files = list(filter(lambda x: '_5.jpg' not in x, training_files))
# for f in training_files:
#     print(f)


validation_files = []
for root, dirs, files in os.walk(validation_image_src):
    for file in files[:100]:
        file_path = os.path.join(root, file)
        validation_files.append(file_path)
        

validation_files = list(filter(lambda x: '_5.jpg' not in x, validation_files))



# for f in validation_files:
#     print(f)

# # Let's sort the files for the heck of it
# from functools import cmp_to_key
# def compare(a, b):
#     candidate_a, frame_a, label_a = os.path.splitext(os.path.basename(a))[0].split('_')
#     candidate_b, frame_b, label_b = os.path.splitext(os.path.basename(b))[0].split('_')
#     if candidate_a != candidate_b:
#         return int(candidate_a) - int(candidate_b)
#     elif label_a != label_b:
#         return int(label_a) - int(label_b)
#     else:
#         return int(frame_a) - int(frame_b)

# training_files = sorted(training_files, key=cmp_to_key(compare))
# validation_files = sorted(validation_files, key=cmp_to_key(compare))

"""
On second thoughts, seems more meaningful for the model to receive data unordered, if not at periods where it
keeps receiving the same labelled data, it might not be learning much. We can save the ordering if we wish to do lstm
"""

import random

training_sample_size = len(training_files)
validation_sample_size = len(validation_files)

# training_sample_size = 25000
# validation_sample_size = 12000

training_files = random.sample(training_files, training_sample_size)
validation_files = random.sample(validation_files, validation_sample_size)

batch_size = 100
training_generator = batch_generator(training_files, batch_size)
validation_generator = batch_generator(validation_files, batch_size)




"""
Now we create our model. Keras allows you to build models in a sequential manner or a functional manner. Sequential
is easier to understand for me. Its only a syntax difference.
"""
print('Creating model...')
model = Sequential()

"""
The model is essentially what we learnt in the course, a series of layers of neurons and in this case, convulutions.

We can tweak the attributes of each layer, such as the size, activation function, etc. This is what they mean by
playing with the parameters.

I believe what is passed between layers are just Numpy arrays, so what happens is that a layer will take in a Numpy
array, transform it using its neurons/convulutions and return the resulting Numpy array.

Note that the input shape and output shape of each layer must match.
"""

# model.add(Conv2D(128, kernel_size=3, activation='relu', input_shape=(100,100,1),  activity_regularizer=l1(0.001)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(100,100,1),  activity_regularizer=l1(0.001)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=3, activation='relu', activity_regularizer=l1(0.001)))
model.add(MaxPooling2D(pool_size=(2, 2)))
# Flattens the 2D data into a 1D Numpy array
model.add(Flatten())
# Dense is your standard MLP layer
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

"""
Conv2D arguments explained:

first argument: number of filters that the layer will learn, each filter will have its own kernel to be convolved
                with the input, each filter will result in a different 2D activation map. All of these maps 
                are passed to the subsequent layer in a NumpyArray
kernel size   : size of the kernel to convolve the input with. Generally smaller kernel means more processing time
                but possibly identify more features (must be odd)
strides       : default=(1, 1) - determines how the kernel is moved along the input matrix in the x and y axis
padding       : default=valid - no padding is added to the activation map
activation    : the activation function to be applied after ecah kernel convulution to introduce non-linearity
input_shape   : required if the layer is used as the first layer of the model, if not it is inferred

https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/
"""

"""
Compiling the model - optimizer is the method used to minimize the loss function e.g. stochastic gradient descent
'adam' seems to be an extension of SGD that is less computationally expensive and more appropriate for images

metrics is just a function used to judge the performance of the model, this is not used in training, and only serves
tool to analyze the effective of the model

https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/
"""
model.compile(optimizer='adam', loss=BinaryCrossentropy(), metrics=['accuracy'])

print('Starting training...')
# Training the model only takes a simple function call
# Epochs is the number of passes over the dataset we want for the training
history = model.fit_generator(training_generator, validation_data=validation_generator, 
                              epochs=10, steps_per_epoch=np.ceil(len(training_files)/batch_size), 
                              validation_steps=np.ceil(len(validation_files)/batch_size), 
                              verbose=1, shuffle=True)


# # Save model to json for future use
# model_json = model.to_json()
# with open("cnn.json", "w") as json_file:
#     json_file.write(model_json)
# # Save weights for future use
# model.save_weights("model.h5")

# You can save model and weights together
model.save("cnn.h5");

# What follows is just a few library calls to plot the results throughout the course of the training
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('cnn_accuracy.png')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('cnn_loss.png')
plt.show()


Using TensorFlow backend.
/home/mlsg/mlsg-venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mlsg/mlsg-venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mlsg/mlsg-venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mlsg/mlsg-venv/lib/python3.7/site-packages/tensorflow/py

Creating generators...
Creating model...

Starting training...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/10
214/214 [==============================] - 1067s 5s/step - loss: 115.1066 - accuracy: 0.6384 - val_loss: 1.8747 - val_accuracy: 0.5505
Epoch 2/10
214/214 [==============================] - 995s 5s/step - loss: 0.9897 - accuracy: 0.8707 - val_loss: 1.4426 - val_accuracy: 0.5502
Epoch 3/10
214/214 [==============================] - 1002s 5s/step - loss: 0.5856 - accuracy: 0.9020 - val_loss: 1.3991 - val_accuracy: 0.5950
Epoch 4/10
214/214 [==============================] - 996s 5s/step - loss: 0.4413 - accuracy: 0.9135 - val_loss: 1.6516 - val_accuracy: 0.5671
Epoch 5/10
214/214 [==============================] - 994s 5s/step - loss: 0.3701 - accuracy: 0.9201 - val_loss: 1.8849 - val_accuracy: 0.6000
Epoch 6/10
214/214 [==============================] - 1001s 5s/step - loss: 0.3255 - accuracy: 0.9279 - val_loss: 1.7380 - 

In [ ]:
print(history)